In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
da_2010_dec = pd.read_excel('Data/rpt.00013060.0000000000000000.DAMLZHBSPP_2010.xlsx', sheet_name='Dec_1')
rt_2010_dec = pd.read_excel('Data/rpt.00013061.0000000000000000.RTMLZHBSPP_2010.xlsx', sheet_name='Dec_1')

In [3]:
zone = "LZ_HOUSTON"
dam_z = da_2010_dec[da_2010_dec["Settlement Point"] == zone].copy()
rtm_z = rt_2010_dec[rt_2010_dec["Settlement Point Name"] == zone].copy()

In [4]:
dam_z

,Delivery Date,Hour Ending,Repeated Hour Flag,Settlement Point,Settlement Point Price
8,12/01/2010,01:00,N,LZ_HOUSTON,35.36
22,12/01/2010,02:00,N,LZ_HOUSTON,30.45
36,12/01/2010,03:00,N,LZ_HOUSTON,29.35
50,12/01/2010,04:00,N,LZ_HOUSTON,29.41
64,12/01/2010,05:00,N,LZ_HOUSTON,30.72
...,...,...,...,...,...
10354,12/31/2010,20:00,N,LZ_HOUSTON,40.35
10368,12/31/2010,21:00,N,LZ_HOUSTON,35.30
10382,12/31/2010,22:00,N,LZ_HOUSTON,30.21
10396,12/31/2010,23:00,N,LZ_HOUSTON,27.08


In [5]:
rtm_z

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
32,12/01/2010,1,1,N,LZ_HOUSTON,LZ,25.08
33,12/01/2010,1,2,N,LZ_HOUSTON,LZ,23.20
34,12/01/2010,1,3,N,LZ_HOUSTON,LZ,23.45
35,12/01/2010,1,4,N,LZ_HOUSTON,LZ,22.17
88,12/01/2010,2,1,N,LZ_HOUSTON,LZ,21.96
...,...,...,...,...,...,...,...
41587,12/31/2010,23,4,N,LZ_HOUSTON,LZ,22.16
41640,12/31/2010,24,1,N,LZ_HOUSTON,LZ,26.41
41641,12/31/2010,24,2,N,LZ_HOUSTON,LZ,25.58
41642,12/31/2010,24,3,N,LZ_HOUSTON,LZ,25.80


In [6]:
dam_z["HE_clean"] = dam_z["Hour Ending"].str.replace("24:00", "00:00")

dam_z["datetime"] = pd.to_datetime(
    dam_z["Delivery Date"] + " " + dam_z["HE_clean"],
    format="%m/%d/%Y %H:%M"
)
mask_24 = dam_z["Hour Ending"] == "24:00"
dam_z.loc[mask_24, "datetime"] += pd.Timedelta(days=1)

dam_z = dam_z.set_index("datetime").sort_index()
dam_z = dam_z.rename(columns={"Settlement Point Price": "DAM"})
dam_hourly = dam_z[["DAM"]]


mask_24 = rtm_z["Delivery Hour"] == 24
rtm_z.loc[mask_24, "Delivery Hour"] = 0
rtm_z.loc[mask_24, "Delivery Date"] = (
    pd.to_datetime(rtm_z.loc[mask_24, "Delivery Date"]) + pd.Timedelta(days=1)
).dt.strftime("%m/%d/%Y")

rtm_z["datetime"] = pd.to_datetime(
    rtm_z["Delivery Date"] + " " + rtm_z["Delivery Hour"].astype(str),
    format="%m/%d/%Y %H"
)

rtm_z = rtm_z.set_index("datetime").sort_index()
rtm_z = rtm_z.rename(columns={"Settlement Point Price": "RTM_15min"})

rtm_hourly = rtm_z["RTM_15min"].groupby("datetime").mean().to_frame("RTM")

In [7]:
dam_hourly

,DAM
datetime,
2010-12-01 01:00:00,35.36
2010-12-01 02:00:00,30.45
2010-12-01 03:00:00,29.35
2010-12-01 04:00:00,29.41
2010-12-01 05:00:00,30.72
...,...
2010-12-31 20:00:00,40.35
2010-12-31 21:00:00,35.30
2010-12-31 22:00:00,30.21


In [8]:
rtm_hourly

,RTM
datetime,
2010-12-01 01:00:00,23.4750
2010-12-01 02:00:00,21.4675
2010-12-01 03:00:00,21.3100
2010-12-01 04:00:00,22.3500
2010-12-01 05:00:00,24.3925
...,...
2010-12-31 20:00:00,31.1775
2010-12-31 21:00:00,27.2700
2010-12-31 22:00:00,25.1375


In [9]:
# federal funds rate in december of 12/1/2010 is 0.18%
fed_funds_df = pd.read_csv('Data/FEDFUNDS.csv')
fed_funds_df

,observation_date,FEDFUNDS
0,10/1/2010,0.19
1,11/1/2010,0.19
2,12/1/2010,0.18


In [10]:
fed_funds_rate = fed_funds_df[fed_funds_df['observation_date'] == '12/1/2010']['FEDFUNDS'].values[0]
fed_funds_rate

0.18

In [11]:
import netCDF4

temp = netCDF4.Dataset('Data/data_stream-oper_stepType-instant.nc')



In [12]:
lat = np.asarray(temp.variables['latitude'])
lon = np.asarray(temp.variables['longitude'])
# temperature 2 meters above ground level
t2m = np.asarray(temp.variables['t2m']) # shape (time, lat, lon)

In [13]:
# time is in hours (31 * 24 = 744 hours)
t2m.shape 

(744, 721, 1440)

In [14]:
# taking the sub grid of interest (around Texas)
temp_sub = t2m[:, 208:257, 1006:1064]
temp_sub.shape

(744, 49, 58)

In [15]:
import yfinance as yf

data = yf.download(
    "SPY",
    start="2010-12-01",
    end="2011-01-01",
    interval="1d"
)
data


/tmp/ipykernel_287150/190695544.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2010-12-01,92.360123,92.535665,91.734263,91.741891,221037200
2010-12-02,93.543144,93.611839,92.451706,92.505132,191213600
2010-12-03,93.795013,93.901867,93.199684,93.222581,151288900
2010-12-06,93.695793,93.909500,93.497348,93.596568,103050500
2010-12-07,93.749222,94.649850,93.695795,94.596423,206581000
2010-12-08,94.092667,94.168990,93.428648,93.863697,138019200
2010-12-09,94.459038,94.657477,93.993459,94.619318,123705100
2010-12-10,95.008575,95.100161,94.436143,94.749070,117571700


In [16]:

ffr = np.full(32, fed_funds_rate, dtype=float)

# 2. Convert daily SPY data to hourly array of length 744


spy_daily = data["Close"]

idx = pd.date_range("2010-12-01", "2010-12-31 23:00", freq="H")



/tmp/ipykernel_287150/1392457149.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range("2010-12-01", "2010-12-31 23:00", freq="H")


In [17]:
spy_daily

Ticker,SPY
Date,
2010-12-01,92.360123
2010-12-02,93.543144
2010-12-03,93.795013
2010-12-06,93.695793
2010-12-07,93.749222
2010-12-08,94.092667
2010-12-09,94.459038
2010-12-10,95.008575
2010-12-13,95.069656


We now have said data: rtm - (744), fed funds - (744), spy - (744), t2m - (744, 49, 58). I would like to predict the average real time value of electricity on the last day.

In [18]:
rtm_daily_mean = rtm_hourly.resample("D").mean()
rtm_daily_min  = rtm_hourly.resample("D").min()
rtm_daily_max  = rtm_hourly.resample("D").max()


In [19]:
rtm_daily_mean.shape

(32, 1)

In [20]:
import numpy as np

# t2m has shape (744, 721, 1440)
n_hours, n_lat, n_lon = t2m.shape
hours_per_day = 24
n_days = n_hours // hours_per_day


t2m_daily = t2m.reshape(n_days, hours_per_day, n_lat, n_lon).mean(axis=1)

print(t2m_daily.shape) 


(31, 721, 1440)


In [21]:
rtm_daily_max = rtm_daily_max[:31]
rtm_daily_mean = rtm_daily_mean[:31]
rtm_daily_min = rtm_daily_min[:31]
ffr = ffr[:31]

making it such that the weekend spy prices stay the same as the previous (market is closed on weekends)

In [22]:

spy_daily_full = spy_daily.asfreq("D").ffill()
idx = pd.date_range("2010-12-01", "2010-12-31", freq="D")
spy_daily_full = spy_daily.reindex(idx).ffill()

In [23]:
print(rtm_daily_max.shape)
print(rtm_daily_mean.shape)
print(rtm_daily_min.shape)
print(spy_daily_full.shape)

print(t2m_daily.shape)
print(ffr.shape)

(31, 1)
(31, 1)
(31, 1)
(31, 1)
(31, 721, 1440)
(31,)
